# vTTS - Universal TTS/STT Serving System 테스트

**vLLM for Speech** - Kaggle에서 vTTS 테스트하기

이 노트북에서는:
- ✅ GitHub에서 vTTS 설치
- ✅ TTS (Text-to-Speech) 테스트
- ✅ STT (Speech-to-Text) 테스트
- ✅ OpenAI API 호환성 테스트

## 🚀 Step 1: 설치

GitHub에서 직접 설치합니다.

In [ ]:
# vTTS 설치 (GitHub에서 직접)
!pip install git+https://github.com/YOUR_USERNAME/vTTS.git --quiet

# 또는 로컬 버전 (개발 중)
# !pip install -e /kaggle/input/vtts --quiet

print("✅ vTTS 설치 완료!")

## 📋 Step 2: GPU 확인

In [ ]:
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

## 🎙️ Step 3: TTS 테스트 (백그라운드 서버)

서버를 백그라운드로 실행하고 클라이언트로 테스트합니다.

In [ ]:
# 서버를 백그라운드로 시작
import subprocess
import time

# Supertonic-2로 TTS 서버 시작 (더미 모델 - 실제로는 CosyVoice나 다른 모델 사용)
# 실제 구현이 없으므로 직접 엔진 테스트로 진행
print("서버 모드는 건너뛰고 직접 엔진 테스트를 진행합니다...")

## 🔧 Step 4: TTS 엔진 직접 테스트

서버 없이 엔진을 직접 사용해봅니다.

In [ ]:
# Faster-Whisper STT 엔진 테스트
from vtts.engines.faster_whisper import FasterWhisperEngine
from vtts.engines.stt_base import STTRequest

# STT 엔진 초기화
print("Loading Faster-Whisper model...")
stt_engine = FasterWhisperEngine(
    model_id="base",  # Kaggle에서는 작은 모델 사용 (빠른 테스트)
    device="cuda" if torch.cuda.is_available() else "cpu"
)
stt_engine.load_model()

print("✅ STT 엔진 로드 완료!")
print(f"지원 언어: {stt_engine.supported_languages[:10]}...")
print(f"번역 지원: {stt_engine.supports_translation}")
print(f"타임스탬프 지원: {stt_engine.supports_timestamps}")

## 🎤 Step 5: 샘플 오디오로 STT 테스트

테스트용 오디오를 생성하고 STT를 실행합니다.

In [ ]:
# gTTS로 테스트 오디오 생성 (TTS 엔진 구현 전 임시)
!pip install gtts --quiet

from gtts import gTTS

# 한국어 샘플 텍스트
text = "안녕하세요, vTTS 테스트입니다. 음성 인식이 잘 작동하는지 확인하고 있습니다."

# 오디오 생성
tts = gTTS(text=text, lang='ko')
tts.save('test_korean.mp3')

print(f"✅ 테스트 오디오 생성 완료: test_korean.mp3")
print(f"원본 텍스트: {text}")

In [ ]:
# STT 실행
import IPython.display as ipd

# 오디오 재생
print("🔊 생성된 오디오:")
ipd.display(ipd.Audio('test_korean.mp3'))

In [ ]:
# Faster-Whisper로 음성 인식
from vtts.engines.stt_base import STTRequest

stt_request = STTRequest(
    audio='test_korean.mp3',
    language='ko',
    task='transcribe',
    response_format='verbose_json',
    timestamp_granularities=['segment']
)

print("🎤 음성 인식 중...")
result = stt_engine.transcribe(stt_request)

print(f"\n✅ 인식 완료!")
print(f"인식된 텍스트: {result.text}")
print(f"감지된 언어: {result.language}")
print(f"신뢰도: {result.metadata['language_probability']:.2%}")

## 📊 Step 6: 타임스탬프 정보 확인

In [ ]:
# 세그먼트 정보 출력
if result.segments:
    print("\n📍 타임스탬프 정보:")
    print("-" * 80)
    for seg in result.segments:
        print(f"[{seg['start']:.2f}s - {seg['end']:.2f}s] {seg['text']}")
        print(f"  - 신뢰도: {seg['avg_logprob']:.2f}")
        print()

## 🌍 Step 7: 다국어 테스트

In [ ]:
# 영어 테스트
english_text = "Hello, this is vTTS testing. We are checking if speech recognition works well."

tts_en = gTTS(text=english_text, lang='en')
tts_en.save('test_english.mp3')

print("🔊 English Audio:")
ipd.display(ipd.Audio('test_english.mp3'))

# STT 실행
stt_request_en = STTRequest(
    audio='test_english.mp3',
    language='en',
    task='transcribe',
    response_format='json'
)

result_en = stt_engine.transcribe(stt_request_en)
print(f"\n✅ English Transcription:")
print(f"Original: {english_text}")
print(f"Transcribed: {result_en.text}")

## 🔄 Step 8: 번역 테스트 (한국어 → 영어)

In [ ]:
# 번역 테스트
stt_request_translate = STTRequest(
    audio='test_korean.mp3',
    task='translate',  # 번역 모드
    response_format='json'
)

print("🌐 한국어 → 영어 번역 중...")
result_translate = stt_engine.transcribe(stt_request_translate)

print(f"\n✅ 번역 완료!")
print(f"원본 (한국어): {text}")
print(f"번역 (영어): {result_translate.text}")

## 📈 Step 9: 성능 벤치마크

In [ ]:
import time

# 성능 측정
num_runs = 5
times = []

print(f"⏱️ 성능 측정 중 ({num_runs}회)...\n")

for i in range(num_runs):
    start = time.time()
    result = stt_engine.transcribe(stt_request)
    elapsed = time.time() - start
    times.append(elapsed)
    print(f"Run {i+1}: {elapsed:.3f}s")

avg_time = sum(times) / len(times)
audio_duration = result.metadata['duration']
rtf = avg_time / audio_duration

print(f"\n📊 벤치마크 결과:")
print(f"평균 처리 시간: {avg_time:.3f}s")
print(f"오디오 길이: {audio_duration:.3f}s")
print(f"RTF (Real-time Factor): {rtf:.3f}x")
print(f"{'✅ 실시간보다 빠름!' if rtf < 1 else '⚠️ 실시간보다 느림'}")

## 🎯 Step 10: 결과 요약

In [ ]:
print("\n" + "="*80)
print("🎉 vTTS 테스트 완료!")
print("="*80)
print()
print("✅ 완료된 테스트:")
print("  - Faster-Whisper 엔진 로드")
print("  - 한국어 음성 인식")
print("  - 영어 음성 인식")
print("  - 한국어 → 영어 번역")
print("  - 타임스탬프 추출")
print("  - 성능 벤치마크")
print()
print("📦 모델 정보:")
print(f"  - 모델: {stt_engine.model_id}")
print(f"  - 디바이스: {stt_engine.device}")
print(f"  - 지원 언어: {len(stt_engine.supported_languages)}개")
print()
print("🚀 다음 단계:")
print("  - GitHub: https://github.com/YOUR_USERNAME/vTTS")
print("  - pip install git+https://github.com/YOUR_USERNAME/vTTS.git")
print("  - vtts serve Supertone/supertonic-2 --stt-model large-v3")
print()

## 💾 Step 11: 결과 저장

In [ ]:
# 결과를 JSON으로 저장
import json

results = {
    "model": stt_engine.model_id,
    "device": stt_engine.device,
    "korean_test": {
        "original": text,
        "transcribed": result.text,
        "language": result.language,
        "confidence": result.metadata['language_probability']
    },
    "english_test": {
        "original": english_text,
        "transcribed": result_en.text
    },
    "translation_test": {
        "original_ko": text,
        "translated_en": result_translate.text
    },
    "performance": {
        "avg_time": avg_time,
        "audio_duration": audio_duration,
        "rtf": rtf
    }
}

with open('vtts_test_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ 결과 저장 완료: vtts_test_results.json")